In [ ]:
import gzip
import json
import string
import collections
from gensim.models import Word2Vec
import torch
import torch.nn as nn
import random
import torch.optim as optim
import math
import nltk
from nltk.corpus import stopwords

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
f = gzip.open('/content/drive/MyDrive/CSE256_summary/cs_dataset/renttherunway_final_data.json.gz')
dataset = []
cnt=0
for l in f:
  dataset.append(json.loads(l))

In [ ]:
dataset[0]

{'fit': 'fit',
 'user_id': '420272',
 'bust size': '34d',
 'item_id': '2260466',
 'weight': '137lbs',
 'rating': '10',
 'rented for': 'vacation',
 'review_text': "An adorable romper! Belt and zipper were a little hard to navigate in a full day of wear/bathroom use, but that's to be expected. Wish it had pockets, but other than that-- absolutely perfect! I got a million compliments.",
 'body type': 'hourglass',
 'review_summary': 'So many compliments!',
 'category': 'romper',
 'height': '5\' 8"',
 'size': 14,
 'age': '28',
 'review_date': 'April 20, 2016'}

In [ ]:
raw_reviews = []
raw_summaries = []
for data in dataset:
  raw_reviews.append(data['review_text'])
  raw_summaries.append(data['review_summary'])
print(raw_reviews[0])
print(raw_summaries[0])

An adorable romper! Belt and zipper were a little hard to navigate in a full day of wear/bathroom use, but that's to be expected. Wish it had pockets, but other than that-- absolutely perfect! I got a million compliments.
So many compliments!


In [ ]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
class wv:
    def __init__(self):
        self.word2index = {"<SOS>":0, "<EOS>":1, "<PAD>":2} # word -> index
        self.word2count = {} # word -> frequency. This is only used for statistical work.
        self.index2word = {0:"<SOS>", 1:"<EOS>", 2:"<PAD>"} # index -> word
        self.n_words = 3 # <sos>, <eos>, <pad>

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [ ]:
alltokens = [['<start>'], ['<end>'], ['<pad>']]
reviews = []
summaries = []
punctuations = set(string.punctuation)
stop_words = set(stopwords.words('english'))

for i in range(len(raw_reviews)):
  r = raw_reviews[i]
  s = raw_summaries[i]

  r = ''.join([c for c in r.lower() if c not in punctuations])
  s = ''.join([c for c in s.lower() if c not in punctuations])
  #print(r)
  #print(s)

  tokens = []
  for word in r.split(' '):
    if (word not in punctuations) and (word != ''):
      #print(word)
      word = word.lower()
      tokens.append(word)
  alltokens.append(list(tokens))
  reviews.append(' '.join(tokens))

  tokens = []
  for word in s.split(' '):
    if (word not in punctuations) and (word != ''):
      #print(word)
      word = word.lower()
      tokens.append(word)
  alltokens.append(list(tokens))
  summaries.append(' '.join(tokens))

In [ ]:
reviews[1506]

'at 59 the long allowed me to wear 25 heels without tripping on the hem the details on the top are very pretty and the skirt with gorgeous pleats and subtle split in the front is simple but elegant i could easily see renting this dress again for a different event it made me feel beautiful and festive we didnt get any good pictures due to the lighting of the event maybe next time the only caveat is that the skirt is big which would be cumbersome for lots of dancing'

In [ ]:
summaries[1506]

'i rented this dress for a formal club xmas party i received so many compliments'

In [ ]:
source = wv()
target = wv()
pairs = []
for s, t in zip(reviews, summaries):
  if len(s.split(' ')) <50: continue
  source.addSentence(s)
  target.addSentence(t)
  full=[s,t]
  pairs.append(full)
  # break

In [ ]:
pairs[100]

['i wore this dress to my school of pharmacy centennial gala it definitely stood out and i received so many compliments from colleagues and strangers many commented that it was their favorite dress of the evening the dress was in excellent condition and the color was gorgeous the back was my favorite part i usually wear a size 2 but the 0 fit like a glove plenty of room and hugs every curve i loved this dress and would recommend it to anyone',
 'absolutely gorgeous compliments all night long']

In [ ]:
len(pairs)

96126

In [ ]:
print(len(source.word2index), len(target.word2index))

50892 11474


In [ ]:
import torch
import torch.nn as nn
import numpy as np
from torch.utils import data
from torch.utils.data import DataLoader
import os
import gensim
import string
from torch.nn import Embedding

class Summary_Loader(data.Dataset):
    def __init__(self, pairs, source, target,
                 review_length = 100, summary_length = 20):
        super().__init__()

        # self.reviews = reviews
        # self.summaries = summaries
        self.pairs = pairs

        self.review_length = review_length
        self.summary_length = summary_length

        self.source = source
        self.target = target

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, index):

        review = self.pairs[index][0].split(' ')
        summary = self.pairs[index][1].split(' ')
        # print(summary)

        review_tensor = np.zeros(shape = (self.review_length))
        summary_tensor = np.zeros(shape = (self.summary_length + 1))

        for i in range(self.review_length):
          if i < len(review):
            review_tensor[i] = self.source.word2index[review[i]]
          else:
            review_tensor[i] = self.source.word2index["<PAD>"]

        summary_tensor[0] = self.target.word2index["<SOS>"]
        for i in range(1, self.summary_length + 1):
          if i - 1 < len(summary):
            summary_tensor[i] = self.target.word2index[summary[i - 1]]
          elif i - 1 == len(summary):
            summary_tensor[i] = self.target.word2index["<EOS>"]
          else:
            summary_tensor[i] = self.target.word2index["<PAD>"]

        return torch.tensor(review_tensor).long(), torch.tensor(summary_tensor).long()


In [ ]:
summary_loader = Summary_Loader(pairs, source, target)

In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, dim_model, dropout_rate, max_len):
        super().__init__()
        # Info
        self.dropout = nn.Dropout(dropout_rate)

        # Encoding - From formula
        pos_encoding = torch.zeros(max_len, dim_model)
        positions_list = torch.arange(0, max_len).float().view(-1, 1)
        division_term = torch.exp(torch.arange(0, dim_model, 2).float() * (-math.log(10000.0)) / dim_model)

        pos_encoding[:, 0::2] = torch.sin(positions_list * division_term)
        pos_encoding[:, 1::2] = torch.cos(positions_list * division_term)

        self.pos_encoding = pos_encoding.unsqueeze(0).transpose(0, 1)

    def forward(self, token_embedding: torch.tensor) -> torch.tensor:
        pos_embeded = token_embedding + self.pos_encoding[:token_embedding.size(0), :]
        return self.dropout(pos_embeded)

In [ ]:
class Transformer(nn.Module):
    """
    Model from "A detailed guide to Pytorch's nn.Transformer() module.", by
    Daniel Melchor: https://medium.com/@danielmelchor/a-detailed-guide-to-pytorchs-nn-transformer-module-c80afbc9ffb1
    """
    # Constructor
    def __init__(
        self,
        num_tokens_input,
        num_tokens_output,
        dim_model,
        num_heads,
        num_encoder_layers,
        num_decoder_layers,
        dropout_p,
    ):
        super().__init__()

        self.dim_model = dim_model

        # LAYERS
        self.positional_encoder = PositionalEncoding(
            dim_model=dim_model, dropout_p=dropout_p, max_len=5000
        )
        self.embedding = nn.Embedding(num_tokens_input, dim_model)

        self.embedding2 = nn.Embedding(num_tokens_output, dim_model)

        self.transformer = nn.Transformer(
            d_model=dim_model,
            nhead=num_heads,
            num_encoder_layers=num_encoder_layers,
            num_decoder_layers=num_decoder_layers,
            dropout=dropout_p,
        )

        self.out = nn.Linear(dim_model, num_tokens_output)

    def forward(self, src, tgt, tgt_mask=None, src_pad_mask=None, tgt_pad_mask=None):

        src = self.embedding(src) * math.sqrt(self.dim_model)
        tgt = self.embedding2(tgt) * math.sqrt(self.dim_model) # Different length for our input and output

        src = self.positional_encoder(src)
        tgt = self.positional_encoder(tgt)
        src = src.permute(1,0,2)
        tgt = tgt.permute(1,0,2)

        transformer_out = self.transformer(src, tgt, tgt_mask=tgt_mask, src_key_padding_mask=src_pad_mask, tgt_key_padding_mask=tgt_pad_mask)
        out = self.out(transformer_out)

        return out

    def get_tgt_mask(self, size) -> torch.tensor:
        mask = torch.tril(torch.ones(size, size) == 1) # Lower triangular matrix
        mask = mask.float()
        mask = mask.masked_fill(mask == 0, float('-inf')) # Convert zeros to -inf
        mask = mask.masked_fill(mask == 1, float(0.0)) # Convert ones to 0

        return mask

    def create_pad_mask(self, matrix: torch.tensor, pad_token: int) -> torch.tensor:
        return (matrix == pad_token)

In [ ]:
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
trainset, valset = torch.utils.data.random_split(summary_loader, [0.95, 0.05], torch.Generator().manual_seed(42))

trainloader = DataLoader(trainset, batch_size = 128, shuffle = True)
valloader = DataLoader(valset, batch_size = 1, shuffle = True)
print(len(trainset), len(valset))

model = Transformer(
    num_tokens_input=len(source.word2index), num_tokens_output = len(target.word2index), dim_model=100, num_heads=2, num_encoder_layers=3, num_decoder_layers=3, dropout_p=0.1
).to(device)

optimizer = optim.Adam(model.parameters())
PAD_ID = 2
criterion = nn.CrossEntropyLoss(ignore_index = PAD_ID)

91320 4806


In [ ]:
! pip install einops

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
def save(path, model):
  ckpt = {
      'model' : model.state_dict(),
  }
  torch.save(ckpt, path)
  print("save {} successfully".format(path))

In [ ]:
from einops import rearrange
def online_eval(model, trainloader):
  model = model.eval()
  tmp=[]
  tmp1=[]
  tmp2 =[]
  with tqdm(total = len(trainloader)) as pbar:
      for i, batch in enumerate(trainloader):
        input, label = batch
        input, label = input.to(device), label.to(device)
        # assert input.shape == (32, 216, 100) and label.shape == (32, 30, 100)

        y_input = label[:,:-1]
        y_expected = label[:,1:]

        # Get mask to mask out the next words
        sequence_length = y_input.size(1)
        tgt_mask = model.get_tgt_mask(sequence_length).to(device)


        pred = model(input, y_input, tgt_mask)
        pred = pred.permute(1, 0, 2)

        # pred = pred.cpu().detach().numpy()

        one_sample = pred[0]

        sf = nn.Softmax(dim = -1)
        one_sample = sf(one_sample)

        #print(one_sample.shape)
        one_sample = torch.argmax(one_sample, dim = -1)

        #print(one_sample.shape)
        #print(one_sample)

        one_sample = one_sample.cpu().detach().numpy()
        word = []
        for i in range(20):
          w = target.index2word[one_sample[i]]
          word.append(w)

        print("translate", word)

        # label = rearrange(label, 'l b -> b l')
        word_label = []
        one_label = label.cpu().detach().numpy()[0]
        #print(one_label.shape)
        for i in range(20):
          w = target.index2word[one_label[i]]
          word_label.append(w)
        print("ground truth", word_label)
        pbar.update(1)
        # print(input.shape, label.shape)
        break

In [ ]:
from tqdm import tqdm
def train(model, iterator, optimizer, criterion, clip = None, valloader = valloader):

    model.train()

    epoch_loss = 0

    for e in range(100):
      with tqdm(total = len(iterator), unit = 'batch') as pbar:
        for i, batch in enumerate(iterator):
            model.train()
            # src = batch.src
            # trg = batch.trg
            src, trg = batch

            src, trg = src.to(device), trg.to(device)
            # print(src.shape, trg.shape)

            optimizer.zero_grad()

            y_input = trg[:,:-1]
            y_expected = trg[:,1:]

            # Get mask to mask out the next words
            sequence_length = y_input.size(1)
            tgt_mask = model.get_tgt_mask(sequence_length).to(device)


            output = model(src, y_input, tgt_mask)

            # print(output.shape)

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]

            # output = output[1:].view(-1, output_dim)
            # trg = trg[1:].view(-1)
            trg = trg.permute(1, 0)
            output = output.reshape(-1, output_dim)
            trg = trg[1:].reshape(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            # print(trg.shape, output.shape)

            loss = criterion(output, trg)

            loss.backward()

            # torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

            optimizer.step()

            epoch_loss += loss.item()

            pbar.set_postfix(loss=loss.item())
            pbar.update(1)
            # break
      print("training evaluation...")
      online_eval(model = model, trainloader = iterator)
      print("validation evaluation...")
      online_eval(model = model, trainloader = valloader)
      print("Finish epoch" + str(e))
      save("/content/drive/MyDrive/CSE256_summary/models/transformerForSummarization_v2.pkl", model)

    # return epoch_loss / len(iterator)

In [ ]:
train(model, trainloader, optimizer, criterion )

100%|██████████| 714/714 [01:25<00:00,  8.38batch/s, loss=3.66]


training evaluation...


  0%|          | 1/714 [00:00<00:30, 23.27it/s]


translate ['i', 'dress', '<EOS>', 'and', '<EOS>', '<EOS>', '<EOS>', '<EOS>', '<EOS>', '<EOS>', '<EOS>', '<EOS>', '<EOS>', '<EOS>', '<EOS>', '<EOS>', '<EOS>', '<EOS>', '<EOS>', '<EOS>']
ground truth ['<SOS>', 'stunning', 'color', 'classy', 'look', '<EOS>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']
validation evaluation...


  0%|          | 1/4806 [00:00<01:00, 78.83it/s]


...

training evaluation...


  0%|          | 1/714 [00:00<00:30, 23.47it/s]


translate ['i', 'compliments', 'told', 'i', 'was', 'the', 'hollywood', 'glamorous', '<EOS>', '<EOS>', '<EOS>', '<EOS>', '<EOS>', '<EOS>', '<EOS>', '<EOS>', '<EOS>', '<EOS>', '<EOS>', '<EOS>']
ground truth ['<SOS>', 'nonstop', 'complimentswas', 'told', 'i', 'was', 'old', 'hollywood', 'glamorous', '<EOS>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']
validation evaluation...


  0%|          | 1/4806 [00:00<00:52, 90.94it/s]


translate ['i', 'dress', '<EOS>', 'for', 'a', 'formal', '<EOS>', '<EOS>', '<EOS>', '<EOS>', '<EOS>', '<EOS>', '<EOS>', '<EOS>', '<EOS>', '<EOS>', '<EOS>', '<EOS>', '<EOS>', '<EOS>']
ground truth ['<SOS>', 'gorgeous', 'jumpsuit', 'perfect', 'for', 'a', 'wedding', '<EOS>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']
Finish epoch99
save /content/drive/MyDrive/CSE256_summary/models/transformerForSummarization_v2.pkl successfully


In [ ]:
! pip install Rouge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from einops import rearrange
from rouge import Rouge
from tqdm import tqdm

def eval_all(model, trainloader):
  model = model.eval()
  tmp=[]
  tmp1=[]
  tmp2 =[]
  score_r, score_p, score_f = 0,0,0
  num = 0
  with tqdm(total = len(trainloader)) as pbar:
      for i, batch in enumerate(trainloader):
        input, label = batch
        input, label = input.to(device), label.to(device)
        # assert input.shape == (32, 216, 100) and label.shape == (32, 30, 100)

        y_input = label[:,:-1]
        y_expected = label[:,1:]

        # Get mask to mask out the next words
        sequence_length = y_input.size(1)
        tgt_mask = model.get_tgt_mask(sequence_length).to(device)


        pred = model(input, y_input, tgt_mask)
        pred = pred.permute(1, 0, 2)

        # pred = pred.cpu().detach().numpy()

        one_sample = pred[0]

        sf = nn.Softmax(dim = -1)
        one_sample = sf(one_sample)

        #print(one_sample.shape)
        one_sample = torch.argmax(one_sample, dim = -1)

        #print(one_sample.shape)
        #print(one_sample)

        one_sample = one_sample.cpu().detach().numpy()
        word = []
        for i in range(20):
          w = target.index2word[one_sample[i]]
          word.append(w)
        #print("translate", word)

        filtered_word = []
        for i in range(len(word)):
          # For LSTM model, the first word is "<SOS>"
          if word[i] != "<EOS>":
            filtered_word.append(word[i])
          else:
            break
        pred_sentence = ' '.join(filtered_word)
        #print(pred_sentence)

        # label = rearrange(label, 'l b -> b l')
        word_label = []
        one_label = label.cpu().detach().numpy()[0]
        #print(one_label.shape)
        for i in range(20):
          w = target.index2word[one_label[i]]
          word_label.append(w)
        #print("ground truth", word_label)

        filtered_label = []
        for i in range(1, len(word_label)):
          # For LSTM model, the first word is "<SOS>"
          if word_label[i] != "<EOS>":
            filtered_label.append(word_label[i])
          else:
            break
        target_sentence = ' '.join(filtered_label)
        #print(target_sentence)

        if pred_sentence == '' or target_sentence == '':
          continue

        rouger = Rouge()
        scores = rouger.get_scores(pred_sentence, target_sentence)
        #print(scores)
        score_r += scores[0]['rouge-1']['r']
        score_p += scores[0]['rouge-1']['p']
        score_f += scores[0]['rouge-1']['f']
        num += 1

        pbar.update(1)
        #print(input.shape, label.shape)
        # break
      print(score_r / num, score_p / num, score_f / num)
      return score_r, score_p, score_f

In [ ]:
ckpt = torch.load("/content/drive/MyDrive/CSE256_summary/models/transformerForSummarization.pkl", map_location='cpu')
model.load_state_dict(ckpt['model'])
valloader = DataLoader(valset, batch_size = 1, shuffle = True)

In [ ]:
eval_all(model, valloader)

100%|█████████▉| 4797/4806 [00:34<00:00, 139.14it/s]

0.29208172067660454 0.37890004507789515 0.3129712164193657


(1401.1160140856718, 1817.5835162386632, 1501.3229251636974)